# Predicting IT activity from ManifoldCLR features

In [1]:
import sys
import os
os.chdir(os.path.dirname(os.getcwd()))
sys.path.append(os.getcwd() + "/src/")
sys.path.append(os.getcwd() + "/neuro_simclr/")

import torch

from main_trial import Backbone, VariationalEncoderConfig, CoefficientEncoder, TransOp_expm

/home/kyle/mambaforge/envs/torch/lib/python3.10/site-packages/brainscore/metrics/__init__.py:37: FutureWarning: xarray subclass Score should explicitly define __slots__
  class Score(DataAssembly):


## Load model

In [7]:
state = torch.load("neuro_results/weights_randomrun.pt", map_location=torch.device('cpu'))
# state = torch.load("neuro_results/model_weights_epoch9999.pt", map_location=torch.device('cpu'))
args = state['args']
args.device = 'cpu'

backbone = Backbone(
    input_dim=args.input_dim,  # TBD
    hidden_dim=args.hidden_dim,
    output_dim=args.backbone_output_dim,
    num_hidden_layers=args.num_hidden_layers,
    norm=args.backbone_batchnorm,
).to(args.device)
backbone.load_state_dict(state['backbone'])

vi_cfg = VariationalEncoderConfig(
    scale_prior=0.001,
    shift_prior=0.005,
    enable_learned_prior=True,
    enable_prior_shift=True,
    enable_prior_warmup=True,
    
    prior_warmup_iters=500,
)
coeff_enc = CoefficientEncoder(vi_cfg, args.backbone_output_dim, args.dict_size, args.threshold).to(
    args.device
)
coeff_enc.load_state_dict(state['coeff_enc'])

transop = TransOp_expm(
    M=args.dict_size,
    N=args.backbone_output_dim,
    stable_init=True,
    real_range=1.0e-4,
    imag_range=5.0,
    dict_count=1,
).to(args.device)
transop.load_state_dict(state['transop'])

<All keys matched successfully>

In [8]:
from datasets import get_dataset
train_data, test_data = get_dataset(
    average_trials=True,
    average_downsample_factor=50,
    random_seed=1830,
    dataset_split_version='stimulus',
)
(v4_train, it_train, label_train, objectid_train, pose_train) = train_data
(v4_test, it_test, label_test, objectid_test, pose_test) = test_data

In [9]:
from evaluation import evaluate_IT_explained_variance, embed_v4_data
backbone.eval()
train_feat = embed_v4_data(v4_train, backbone, args.device)
test_feat = embed_v4_data(v4_test, backbone, args.device)
evaluate_IT_explained_variance(train_feat, it_train, test_feat, it_test, args)

0.14063525

In [10]:
evaluate_IT_explained_variance(v4_train, it_train, v4_test, it_test, args)

0.2803682